# Data Preparation and Exploration

Data **preparation** and **exploration** are the linchpins in any data analysis project, laying the critical groundwork for applying any quantitative modeling. Data preparation ensures the quality and accuracy of data, minimizing potential inaccuracies in subsequent analyses. This process includes tasks such as cleaning, handling missing values, and standardization, all designed to increase the reliability of results.

On the other hand, data exploration facilitates a deeper understanding of the data's underlying structure and characteristics, informing the choice of appropriate models and guiding hypothesis formation. This step is key to identifying patterns, relationships, and anomalies.

Fitting any models to you data risks yielding misleading or false insights without a solid foundation of well-prepared and thoroughly understood data. Despite being less glamorous than modeling itself, the importance of these preliminary steps cannot be overstated.

The snippets of code provided serve multiple purposes for our data analysis pipeline:
- Firstly, they allow for loading data obtained from various sources. 
- Following this, we can specify the time frame for our analysis. 
- The data is then aggregated for consistency and ease of analysis. 
- After aggregation, the code assists in identifying and removing missing values for a clean dataset. 

An important manual intervention worth noting is the replacement of "." with NaN, specifically for data procured from the [FRED database](https://fred.stlouisfed.org/), to ensure the correct identification and handling of missing data points.

**REMARK**: You can find the files we will use here on the Canvas website for our class.

## Import Modules and Load Files

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from scipy.stats import pearsonr
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Access individual DataFrame as: data['^GSPC'], data['GOLDPMGBD228NLBM'], etc.
files = ["^GSPC", "GOLDPMGBD228NLBM", "DCOILWTICO", "DGS3MO", "DGS1"]
data_dir = "data/to_merge_lect4/"
data = {f: pd.read_csv(data_dir + f + ".csv", index_col=0) for f in files}

In [ ]:
for d in data:
    print(data[d].shape)

In [ ]:
sp = data["^GSPC"]
gld = data["GOLDPMGBD228NLBM"]
wti = data["DCOILWTICO"]
bill3m = data["DGS3MO"]
bill1y = data["DGS1"]

---
**Question:**  
*Do you already have a sense of what those variables represents?*
---

<details>
  <summary>Click to expand!</summary>
  
1. `sp = data["^GSPC"]`
   - S&P 500 index, which is a stock market index that measures the stock performance of 500 large companies listed on stock exchanges in the United States. The symbol `^GSPC` is commonly used to denote this index.
   
2. `gld = data["GOLDPMGBD228NLBM"]`
   - Gold price. The code `GOLDPMGBD228NLBM` is used to denote the price of gold per troy ounce in the London market, usually updated on business days by the Federal Reserve Bank of St. Louis.

3. `wti = data["DCOILWTICO"]`
   - West Texas Intermediate (WTI) crude oil prices. `DCOILWTICO` is a commonly used code to represent the daily prices of WTI crude oil.

4. `bill3m = data["DGS3MO"]`
   - 3-month Treasury bill rate. The code `DGS3MO` is used to denote the daily interest rate on a 3-month Treasury bill, which is a short-term debt obligation issued by the U.S. government.

5. `bill1y = data["DGS1"]`
   - 1-year Treasury bill rate. The code `DGS1` is used to denote the daily interest rate on a 1-year Treasury bill, another type of short-term debt obligation issued by the U.S. government.

</details>


In [ ]:
sp.head()

In [ ]:
sp_close_vol = sp[["Adj Close", "Volume"]]

In [ ]:
print(sp_close_vol.head())
print("----------------------------------")
print(gld.head())
print("----------------------------------")
print(wti.head())
print("----------------------------------")
print(bill3m.head())
print("----------------------------------")
print(bill1y.head())

In [ ]:
print(sp_close_vol.tail())
print("----------------------------------")
print(gld.tail())
print("----------------------------------")
print(wti.tail())
print("----------------------------------")
print(bill3m.tail())
print("----------------------------------")
print(bill1y.tail())

In [ ]:
rv_df = pd.read_excel("{}/intradayRV.xlsx".format(data_dir), index_col=0)

In [ ]:
rv_df.index = pd.to_datetime(rv_df.index)

In [ ]:
rv_df.head()

In [ ]:
rv_df.tail()

The following function, `add_overnight()`, corrects for overnight bias in financial data, typically realized volatility estimates. It does this by scaling the Close-to-Close Realized Volatility (CRV) by a bias correction factor.

Here's a step-by-step breakdown of what the function does:

- `meanAbsRet = np.sqrt(((returns**2).mean())*252)`: This line calculates the root mean square of the daily returns, scaled up to annual returns by multiplying with 252 (the typical number of trading days in a year). This gives an annualized measure of the average absolute return.

- `meanCRV = np.sqrt((CRV**2).mean())`: This line calculates the root mean square of the Close-to-Close Realized Volatility.

- `biasCorrFactor = meanAbsRet/meanCRV`: This calculates the bias correction factor by taking the ratio of meanAbsRet to meanCRV. If meanAbsRet is greater than meanCRV, it indicates that the CRV might be underestimating the actual volatility (likely due to ignoring overnight return movements), and the biasCorrFactor will be greater than 1. Conversely, if meanAbsRet is less than meanCRV, the biasCorrFactor will be less than 1, indicating a potential overestimation by the CRV.

- `CRV = CRV*biasCorrFactor`: The CRV is then scaled by this biasCorrFactor to adjust for the detected bias.

The function returns the corrected CRV.

In [ ]:
def add_overnight(CRV, returns):
    meanAbsRet = np.sqrt(((returns**2).mean()) * 252)

    meanCRV = np.sqrt((CRV**2).mean())

    biasCorrFactor = meanAbsRet / meanCRV

    CRV = CRV * biasCorrFactor

    return CRV

In [ ]:
rv_df["CRV_over"] = add_overnight(rv_df["CRV"], rv_df["Return_close"])

In [ ]:
rv_df.head()

In [ ]:
rv_df["CRV_over"] = rv_df["CRV_over"] / np.sqrt(252)

In [ ]:
rv_df.head()

In [ ]:
RealVol = rv_df[["CRV_over", "Return_close"]]

In [ ]:
RealVol.head()

## Period selection

In [ ]:
start_date = "1986-01-02"
end_date = "2009-02-04"

sp_close_vol = sp_close_vol.loc[start_date:end_date].copy()
sp_close_vol.index = pd.to_datetime(sp_close_vol.index)

gld_price = gld.loc[start_date:end_date].copy()
gld_price.index = pd.to_datetime(gld_price.index)

wti_price = wti.loc[start_date:end_date].copy()
wti_price.index = pd.to_datetime(wti_price.index)

bill3m_rate = bill3m.loc[start_date:end_date].copy()
bill3m_rate.index = pd.to_datetime(bill3m_rate.index)

bill1y_rate = bill1y.loc[start_date:end_date].copy()
bill1y_rate.index = pd.to_datetime(bill1y_rate.index)

RealVol = RealVol.loc[start_date:end_date].copy()

In [ ]:
print(sp_close_vol.head(1))
print("----------------------------------")
print(gld_price.head(1))
print("----------------------------------")
print(wti_price.head(1))
print("----------------------------------")
print(bill3m_rate.head(1))
print("----------------------------------")
print(bill1y_rate.head(1))
print("----------------------------------")
print(RealVol.head(1))

In [ ]:
print(sp_close_vol.tail(1))
print("----------------------------------")
print(gld_price.tail(1))
print("----------------------------------")
print(wti_price.tail(1))
print("----------------------------------")
print(bill3m_rate.tail(1))
print("----------------------------------")
print(bill1y_rate.tail(1))
print("----------------------------------")
print(RealVol.tail(1))

In [ ]:
print(sp_close_vol.info())
print("----------------------------------")
print(gld_price.info())
print("----------------------------------")
print(wti_price.info())
print("----------------------------------")
print(bill3m_rate.info())
print("----------------------------------")
print(bill1y_rate.info())
print("----------------------------------")
print(RealVol.info())

In [ ]:
data = list((sp_close_vol, gld_price, wti_price, bill3m_rate, bill1y_rate, RealVol))

for df in data:
    print(len(df))
    print("--------------------------------------------")

### Drop Calendar Holidays

Check which dates are holidays and drop them from the dataframe

In [ ]:
cal = calendar()
holidays = cal.holidays(start=start_date, end=end_date)

In [ ]:
def drop_holidays(df, holidays):
    df["Holiday"] = df.index.isin(holidays)
    df = df.drop(df[df["Holiday"] == True].index)
    df.drop("Holiday", axis=1, inplace=True)
    return df


sp = drop_holidays(sp_close_vol, holidays)
gld_price = drop_holidays(gld_price, holidays)
wti_price = drop_holidays(wti_price, holidays)
bill3m_rate = drop_holidays(bill3m_rate, holidays)
bill1y_rate = drop_holidays(bill1y_rate, holidays)
RealVol = drop_holidays(RealVol, holidays)

In [ ]:
data = list((sp_close_vol, gld_price, wti_price, bill3m_rate, bill1y_rate, RealVol))
for df in data:
    print(len(df))
    print("--------------------------------------------")

## Merge Dataframes

### Merge bill rates and crude oil price

In [ ]:
bill_rates = pd.merge(
    left=bill3m_rate, right=bill1y_rate, left_index=True, right_index=True
)

In [ ]:
bill_rates.shape

In [ ]:
bill_rates.head()

In [ ]:
print(len(bill1y_rate))
print("________________________________")
print(len(bill3m_rate))
print("________________________________")
print(len(bill_rates))
print("________________________________")

In [ ]:
bill_wti = bill_rates.merge(wti_price, left_index=True, right_index=True)

In [ ]:
bill_wti.head()

In [ ]:
print(len(bill_rates))
print("________________________________")
print(len(wti_price))
print("________________________________")
print(len(bill_wti))
print("________________________________")

### Merge with RV

In [ ]:
bill_wti_rv = bill_wti.merge(RealVol, left_index=True, right_index=True)

In [ ]:
bill_wti_rv.head()

In [ ]:
print(len(bill_wti))
print("________________________________")
print(len(RealVol))
print("________________________________")
print(len(bill_wti_rv))
print("________________________________")

### Merge with gold and obtain final df

In [ ]:
df = bill_wti_rv.merge(gld_price, left_index=True, right_index=True, how="inner")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
print(len(bill_wti_rv))
print("________________________________")
print(len(gld_price))
print("________________________________")
print(len(df))
print("________________________________")

In [ ]:
df_spy = df.merge(sp_close_vol, left_index=True, right_index=True, how="inner")

In [ ]:
df_spy.head()

In [ ]:
df = df_spy.rename(
    columns={
        "DCOILWTICO": "Oil",
        "DGS3MO": "TBill3M",
        "DGS1": "TBill1Y",
        "CRV_over": "RV",
        "GOLDPMGBD228NLBM": "Gold",
        "Adj Close": "SP_close",
        "Volume": "SP_volume",
    }
)

In [ ]:
df["weekday"] = df.index.dayofweek

In [ ]:
df.info()

In [ ]:
df

## Exploratory Data Analysis : A first look at the (time-series) data

Let's observe where we have missing data in our dataset

In [ ]:
df.isnull().sum()

Everything seems to be perfect, but...

In [ ]:
df.dtypes

In [ ]:
pd.to_numeric(df["Gold"], errors="coerce").isna().sum()

In [ ]:
# Create a function that tries to convert a value to float and returns a boolean result
def is_float(x):
    try:
        float(x)
        return True
    except:
        return False


# Apply this function to each element of 'Gold' column
mask = df["Gold"].apply(is_float)

# Print the rows in 'Gold' where the value could not be converted to float
print(df["Gold"][~mask])

In [ ]:
df["Gold"] = pd.to_numeric(df["Gold"], errors="coerce")
df["Oil"] = pd.to_numeric(df["Oil"], errors="coerce")

In [ ]:
df.dtypes

In [ ]:
df[["TBill3M", "TBill1Y"]] = df[["TBill3M", "TBill1Y"]].astype(float)

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df = df.fillna(method="ffill")

In [ ]:
df.to_csv("data/Rv_daily_lec4.csv")

In [ ]:
df.isnull().sum()

We can plot all the time series in the same figure to have a close look at their trends

In [ ]:
df.plot(figsize=(10, 5))

The above plot is not easily interpretable because time series are on different scale! There's a quick solution though

In [ ]:
df.plot(logy=True, figsize=(10, 5))

### Additional Measures and Plots that helps with the EDA

The first step in data analytics is learning about your datasets' elementary characteristics. In terms of statistical science, we're talking about **Descriptive statistics** which includes: 

* measuring *sample mean, median, and mode*, as measures of central tendency; 
* learning about the probability distributions of relevant variables from the data set; 
* learning about dispersion by inspecting sample variances (or standard deviations), ranges, IQRs, and similar; 
* inspecting the data visually, of course. 

The latter procedures have come to be known as **Exploratory Data Analysis (EDA)** in the course of the second half of the 20th century; the term was popularised by the famous American statistician John Tukey (1915-2000), who published a book on the topic titled "Exploratory Data Analysis" in 1977.


There seems to be a lot of observation....but we don't have such observation for all the variables. We will consider only a piece of the dataframe for this reason.

In [ ]:
df.describe().transpose()

In [ ]:
var = "SP_close"  # try also('displacement', 'horsepower', 'weight', 'acceleration')
df[df[var] == df[var].min()]

In [ ]:
df[df[var] == df[var].max()]

### Box Plot

**Box Plot** is the visual representation of the depicting groups of numerical data through their quartiles. Boxplot is also used to detect the outlier in the dataset. It captures the summary of the data efficiently with a simple box and whiskers and allows us to compare easily across groups. Boxplot summarizes sample data using 25th, 50th, and 75th percentiles. These percentiles are also known as the lower, median, and upper quartiles.

A box plot consists of 5 things:
 
* First Quartile or 25%
* Median (Second Quartile) or 50%
* Third Quartile or 75%
* IQR (Inter Quantile Range)
* Outliers

In [ ]:
Q1 = df["SP_close"].quantile(0.25)  # Compute the quantile
Q1

In [ ]:
df["SP_close"].quantile([0.25, 0.50, 0.75])  # A sequence of quantiles

In [ ]:
# Inter quantile Range
Q3 = df["SP_close"].quantile(0.75)
IQR = Q3 - Q1
IQR

In [ ]:
# Box Plot using pandas or seaborn
for column in df.columns:
    if df[column].dtype == "float64":
        fig, ax = fig, ax = plt.subplots(figsize=(8, 3))
        sns.boxplot(x=df[column], ax=ax)

The box extends from the Q1 to Q3 quartile values of the data, with a line at the median (Q2). The whiskers extend from the edges of box to show the range of the data. The position of the whiskers is set by default to:

* Upper whisker = $min(max(x), Q3 + 1.5*IQR)$
* Lower whisker = $max(min(x), Q1 - 1.5*IQR)$ 

Outlier points are those past the end of the whiskers.

### Histogram and Kernel density

In [ ]:
# Histogram using Pandas
df["SP_close"].plot.hist(density=1, bins=100)  # choose number of bins

In [ ]:
# Histogram using Pandas
df["TBill1Y"].plot.hist(density=1, bins=100)  # choose number of bins

In [ ]:
# Kernel density estimation
df["SP_close"].plot.density(bw_method=0.9)

In [ ]:
# Histogram using Pandas
df["TBill1Y"].plot.density(bw_method=0.9)

In [ ]:
df.columns

In [ ]:
sns.displot(df["Return_close"])  # using seaborn

In [ ]:
print("Skewness: %f" % df["Return_close"].skew())
print("Kurtosis: %f" % df["Return_close"].kurt())

Skewness and kurtosis are two important statistical concepts that describe the shape of a distribution.

Skewness refers to a distribution's symmetry, or more precisely, the lack thereof. A skewness of 0 means the distribution is perfectly symmetrical around the mean, while a negative skew indicates that the left tail is longer or fatter than the right. In your case, a skewness of -3.093178 means that the 'Return_close' distribution is heavily skewed to the left.

Kurtosis, on the other hand, refers to the "tailedness" of the distribution. A kurtosis of 3 (or 0, depending on the definition used) is expected from a normal distribution. If the kurtosis is higher than 3 (or 0), the distribution is said to be leptokurtic, which means it has heavier tails or a sharper peak than the normal distribution. The value of 94.653784 for kurtosis in your case is significantly larger than 3, suggesting a leptokurtic distribution with extremely heavy tails and a sharp peak. This indicates a high probability of extreme values or outliers in your 'Return_close' distribution.

### Preprocessing techniques

For the majority of machine learning models, but also standard econometrics models, it is often convenient to scale the data to the same range of [0,1]


In general, learning algorithms benefit from the standardization of the data set.
If some outliers are present in the set, robust scalers or transformers are more appropriate. The `preprocessing` module of the `sklearn` package provides several standard utility functions and transformer classes to change raw feature vectors into a representation that is more suitable for the estimators we will use. In the following classes, we will see a different use of these scalers. 

**REMARK** : *Standardization of datasets is a common requirement for many machine learning estimators that might misbehave if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance.*

In practice, we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

**The reasons are the following**: many elements used in the objective function of a learning algorithm (such as the RBF kernel of Support Vector Machines or the $L_1$ and $L_2$ regularizers of linear models) assume that all features are centered around zero and have variance in the same order. Suppose a feature has a larger order of magnitude variance than others. In that case, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.


Before using `sklearn`, we can define a function `scale` to look at the calculation.

In [ ]:
def standard_scale(a):
    b = (a - a.mean()) / a.std()
    return b

In [ ]:
df.head()

In [ ]:
df_scale = (
    df.copy()
)  # remember that otherwise data_scale will be another name for the same object

In [ ]:
df_scale = df_scale.apply(standard_scale)

In [ ]:
df_scale.head()

All our data is now scaled to resemble a Gaussian distribution with unit variance. This will help us visualize data better and help the model to ingest them. We used a copy of the original data-set for this as we will use the original dataset later when we build regression models.


### Other possible Scalers

In [ ]:
# Standard scaler : standardize column with mean and standard deviation
data_scaled = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)
data_scaled.describe()

### If there are many outliers, we can use a robust routine

In [ ]:
data_robust = pd.DataFrame(RobustScaler().fit_transform(df), columns=df.columns)
data_robust.describe()

## Visualize some relationships

In [ ]:
df.head()

In [ ]:
var = "RV"
data_plt = pd.concat([np.sign(df["Return_close"]), df[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x="Return_close", y=var, data=data_plt)

In [ ]:
df["Return_close"].iloc[np.where(np.sign(df["Return_close"]) == 0)]

## Correlations

**REMINDER** Given two random variables, $X$ and $Y$, their (sample) covariance is given by:
$$Cov(X,Y)= \mathbb{E}\left[(X-\mathbb{E}[X])(Y-\mathbb{E}[Y]) \right]= \frac{1}{N-1}\sum_{i=1}^N (X_i-\hat{X})(Y_i-\hat{Y}) $$
and the Pearson correlation is just the covariance of standardized variables:
$$ \rho(X,Y)=\frac{Cov(X,Y)}{\sigma(X)\sigma(Y)}$$

In [ ]:
df.columns

In [ ]:
var = "RV"
plot = sns.lmplot(y=var, x="Return_close", data=df)

In [ ]:
# data[['mpg',var]].corr()   #compute Pearson correlation
np.corrcoef([df["Return_close"], df["RV"]])

In [ ]:
pearsonr(df["Return_close"], df["RV"])

### Robust correlation (Spearman correlation)
In presence of outliers one can use Robust Statistics: the Spearman correlation is defined as 
$$\hat{\rho}(X,Y)= \rho(Rank(X),Rank(Y))$$

In [ ]:
plot = sns.lmplot(y=var, x="Return_close", data=df.rank())
# Equal values are assigned a rank that is the average of the ranks of those values.

In [ ]:
df[["RV", "Return_close"]].corr(method="spearman")  # compute Pearson correlation

In [ ]:
df.corr()

[Colormap choices](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
# Compute the correlation matrix
corr = df.corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(10, 7))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.3,
    center=0,
    square=True,
    annot=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

## Statistical tests

In finance we often work with time series. Before starting any modeling task, we need to understand if these time series respect some properties.


### Dickey-Fuller Test for Stationarity

The Dickey-Fuller test, specifically the Augmented Dickey-Fuller (ADF) test, is a statistical procedure used to test whether a time series is stationary. A time series is said to be stationary if its statistical properties such as mean and variance remain constant over time. The hypothesis behind the Dickey-Fuller test can be detailed as follows:

#### Null Hypothesis (H0):
The null hypothesis of the test is that the time series is non-stationary, implying that it has a unit root and the time series follows a random walk model. Mathematically, it can be represented as:


$$[ H_0: \delta = 0 \, (\text{Unit Root Present}) ]$$

#### Alternative Hypothesis (H1):
The alternative hypothesis is that the time series is stationary, meaning it does not have a unit root and the time-dependent structure in the series is stable over time. Mathematically, it can be represented as:

$$[ H_1: \delta < 0 \, (\text{No Unit Root}) ]$$

#### Test Statistic:
The test statistic is calculated based on the coefficients of an autoregressive model fitted to the data. The formula for the test statistic is given by:

$$[ \text{ADF Statistic} = \frac{(\hat{\delta} - 0)}{\text{Standard Error of } \hat{\delta}} ]$$

#### Critical Values:
The test statistic is then compared with critical values at different significance levels (usually 1%, 5%, and 10%) to decide whether to reject or fail to reject the null hypothesis. If the test statistic is less than the critical value, we reject the null hypothesis in favor of the alternative hypothesis, indicating stationarity.

#### Implementation:
In practice, the test is implemented by estimating the following regression model:

$$[ \Delta Y_t = \alpha + \beta t + \gamma Y_{t-1} + \delta_1 \Delta Y_{t-1} + \delta_2 \Delta Y_{t-2} + \ldots + \delta_p \Delta Y_{t-p} + \varepsilon_t ]$$

Where:
- $ΔY_t$: The difference in the series at time $t$
- $α$: Constant term
- $βt$: Trend component
- $γ$: Coefficient of the lagged level of the series
- $δ_i$: Coefficients of the lagged differences of the series
- $ε_t$: Error term at time $t$


The Dickey-Fuller test is a powerful tool for analyzing the stationarity of time series data, which is a crucial assumption in many time series modeling techniques, including ARIMA and GARCH models.



In [ ]:
df.columns

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
df["Return_close"].plot(ax=ax)

In [ ]:
# Select only numeric columns from df
df_numeric = df.select_dtypes(include=[np.number])

# ADF statistic to check stationarity
for col in df_numeric.columns:
    timeseries = df_numeric[col].dropna()
    result = adfuller(timeseries)
    plt.figure(figsize=(8, 4))
    ax = timeseries.plot(secondary_y=False, logy=False)

    print(
        f"Testing {col} from {timeseries.index[0]:%Y-%m-%d} to {timeseries.index[-1]:%Y-%m-%d} for Stationarity"
    )
    print(f"ADF Statistic: {result[0]:.3f}")
    print(f"p-value: {result[1]:.3E}")

    if result[0] > result[4]["5%"]:
        conclusion = (
            f"Failed to Reject H_0 at 5% -> {col} Time Series is Non-Stationary"
        )
    else:
        conclusion = f"Reject H_0 at at 5% -> {col} Time Series is Stationary"
    print(conclusion)

    ax.text(
        x=timeseries.index[-1],
        y=timeseries.max(),
        s=f"Testing {col}\nADF Statistic: {result[0]:.3f}\np-value: {result[1]:.3E}\n"
        + conclusion,
        horizontalalignment="right",
        verticalalignment="top",
        bbox=dict(facecolor="white", alpha=0.5, boxstyle="round,pad=0.5"),
    )

    print("\n")

#### Differentiation

If we calculate the differences of a time series (often enough) we can obtain a stationary time series but we throw away information about the absolute level and recent development.

In [ ]:
# SP_close
plt.figure(figsize=(8, 4))
df["SP_close"].plot(label="SP_close")
plt.legend(loc="upper left")
plt.title("SP Close")
plt.xlabel("Index")
plt.ylabel("SP Close Value")

# Differences in SP_close
plt.figure(figsize=(8, 4))
df["SP_close"].diff().plot(alpha=0.5, label="Differences")
plt.legend(loc="upper left")
plt.title("Differences in SP Close")
plt.xlabel("Index")
plt.ylabel("Difference")


# Return_close
plt.figure(figsize=(8, 4))
df["Return_close"].plot(alpha=0.5, label="Percent Change")
plt.legend(loc="upper left")
plt.title("Percent Change (Return Close)")
plt.xlabel("Index")
plt.ylabel("Return Close")

# Log Differences
plt.figure(figsize=(8, 4))
np.log(df["SP_close"]).diff().plot(alpha=0.5, label="Log Differences")
plt.legend(loc="upper left")
plt.title("Log Differences in SP Close")
plt.xlabel("Index")
plt.ylabel("Log Difference")


### Fractional Differentiation of Financial Time Series

Fractional differentiation is a mathematical technique used in the analysis of time series data, particularly in the field of finance. This technique allows for the transformation of non-stationary time series data into stationary, while maintaining memory properties, which is often lost in the process of regular differentiation.

In financial time series analysis, fractional differentiation can be a powerful tool for modeling and forecasting, as it helps in retaining as much information as possible about the original series, which can be crucial for predicting future movements.

The process of fractional differentiation is mathematically complex, involving the application of fractional calculus. The fractional differencing operator is defined as:

$$[
D^{d}X_t = \sum_{k=0}^{\infty} \binom{d}{k} (-1)^k X_{t-k}
]$$

Where:
- $D^{d}$: Fractional differencing operator
- $X_t$: Time series data at time $t$
- $d$: Order of differentiation, which is a fractional number
- $k$: Lag operator

The choice of the differentiation order $d$ is critical, as it determines the balance between removing noise and retaining memory in the series.

#### References
1. Hosking, J. R. M. (1981). Fractional differencing. Biometrika, 68(1), 165-176.
2. Granger, C. W. J., & Joyeux, R. (1980). An introduction to long-memory time series models and fractional differencing. Journal of Time Series Analysis, 1(1), 15-29.
3. Hurst, H. E. (1951). Long-term storage capacity of reservoirs. Transactions of the American Society of Civil Engineers, 116, 770-799.
4. Walasek, R., & Gajda, J. (Year of Publication). Fractional differentiation and its use in machine learning.  International Journal of Advances in Engineering Sciences and Applied Mathematics 13.2-3 (2021): 270-277.


By understanding and applying fractional differentiation, financial analysts and researchers can develop more accurate and informative models for financial time series analysis, enhancing the predictive power and reliability of their analyses.


In [ ]:
def fractional_difference(series, d, lag_cutoff=1e-5):
    """
    Apply fractional differentiation to a time series.
    series: pandas Series
    d: Fractional order of differentiation
    lag_cutoff: Threshold to truncate weights
    """
    weights = [1.0]  # Initialize the weights
    for k in range(1, len(series)):
        weight = -weights[-1] * (d - k + 1) / k
        if abs(weight) < lag_cutoff:  # Stop if weight is too small
            break
        weights.append(weight)
    weights = np.array(weights[::-1])  # Reverse weights for convolution

    # Apply fractional differencing
    diff_series = np.convolve(series, weights, mode="valid")
    return pd.Series(diff_series, index=series.index[len(weights) - 1:])

In [ ]:
orders

In [ ]:
# Parameters for differentiation
orders = [0.2, 0.5, 0.8]
alphas = orders[::-1]# Fractional differentiation orders

# Generate fractionally differentiated series
frac_diff_series = {f"d={d}": fractional_difference(df["SP_close"], d) for d in orders}

# Plot the original series
plt.figure(figsize=(10, 5))
plt.plot(df["SP_close"], label="Original SP_close", linewidth=2, color="blue")
plt.title("Original SP_close Series", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("SP_close", fontsize=12)
plt.legend()

# Plot the fractionally differentiated series
plt.figure(figsize=(10, 5))
for i,(d, series) in enumerate(frac_diff_series.items()):
    plt.plot(series, label=f"Fractional Diff ({d})", linewidth=2, alpha=alphas[i])
plt.title("Fractionally Differentiated SP_close Series", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Value", fontsize=12)
plt.legend()

#### Why Are Observations Lost in Fractional Differentiation?

Fractional differentiation requires a weighted sum of past values in the series. The process truncates these weights when they become negligibly small, which leads to the loss of initial observations.

The number of observations lost depends on:
- The fractional order ($d$): Lower $d$ retains memory longer, requiring more past values.
- The cutoff threshold: Smaller thresholds retain more terms but lose more observations.

To reduce data loss, you can adjust the cutoff threshold or use higher $d$ values, though this may affect the balance between memory retention and stationarity.
